In [219]:
##################################################
# Make the app
##################################################

title_html = """
<h2>Bias Detection Engine</h2>

<ul style="line-height: 1.5">
  <li>This bias detection engine allows for the detection of bias in facial classification and text classification models</li>

</ul>
"""

description_html = """
<style>
p {
    margin-bottom: 1.2em;
    line-height: 1.5;
}
</style>

<p>Complete the form below to set up the correct parameters and run our engine over your model</p>

<p>Source code <a href="https://github.com/ajfrai/Pytorch-BiasDetection"> here</a>. </p>
"""

In [ ]:
!pip install stl
!pip install ipyvolume
# !pip install ipyvuetify
# !pip install bqplot

In [201]:
submit = widgets.Button(
    description='Submit to Engine',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Submit to Engine',
    icon='check'
)

In [202]:
 trans = widgets.Button(
    description='Transform',
    disabled=False,
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Check Transformations',
    icon='check'
)

In [203]:
import pylab as plt
import numpy as np

def plot_image_view():
  Z = np.random.random((500,500))   # Test data
  plt.imshow(Z, cmap='gray', interpolation='nearest')
  plt.show()

In [204]:
image_view = widgets.interactive(plot_image_view)


In [205]:
def update_plt(self):
  image_view = widgets.interactive(plot_image_view)

In [218]:
def img_update(self):
  global resize_width, resize_height, centercrop, normalize
  resize_width = widgets.IntText(value='224',
    placeholder='Type something',
    description='Resize Width:',
    disabled=False, layout=dict(width='auto', height='auto'))
  
  resize_height = widgets.IntText(value='224',
    placeholder='Type something',
    description='Height:',
    disabled=False, layout=dict(width='auto', height='auto'))

  centercrop = widgets.IntText(value='224',
    placeholder='Type something',
    description='Center Crop:',
    disabled=False, layout=dict(width='auto', height='auto'))
  
  normalize = widgets.Text(
    placeholder='ex. [[0.5,0.5,0.5],[0.5,0.5,0.5]]',
    description='Normalize:',
    disabled=False, layout=dict(width='auto', height='auto'))
  
  trans.on_click(update_plt)

  vbox.children = (widgets.HTML(title_html, layout=widgets.Layout(margin='0 0 3em 0', max_width='800px')),
            widgets.HTML(description_html, layout=widgets.Layout(margin='3em 0 0 0', max_width='800px')),widgets.HBox([file_picker, file_name])
  ,widgets.HBox([img,txt])
  ,widgets.HBox([resize_width, resize_height])
  ,widgets.HBox([centercrop]),widgets.HBox([normalize])
  ,widgets.HBox([image_view]), submit)


def txt_update(self):
  global maxLength, tokenizer
  
  tokenizer = widgets.Dropdown(
    options=['T5', 'T5', 'T5'],
    value='T5',
    description='Tokenizer:',
    disabled=False)
  
  maxLength = widgets.IntText(value='512',
    placeholder='Type something',
    description='Text Length:',
    disabled=False, layout=dict(width='auto', height='auto'))
  
  vbox.children = (widgets.HTML(title_html, layout=widgets.Layout(margin='0 0 3em 0', max_width='800px')),
            widgets.HTML(description_html, layout=widgets.Layout(margin='3em 0 0 0', max_width='800px')),widgets.HBox([file_picker, file_name]),widgets.HBox([img,txt]),widgets.HBox([maxLength, tokenizer]), submit)

# get filename from FileUpload dict
def get_name(file_upload):
    if len(file_upload) == 0:
        return ''
    return next(iter(file_upload.values()))['metadata']['name']

def change_input(change):
    global vbox, file_picker, file_name
    vbox.children = (widgets.HTML(title_html, layout=widgets.Layout(margin='0 0 3em 0', max_width='800px')),
            widgets.HTML(description_html, layout=widgets.Layout(margin='3em 0 0 0', max_width='800px')),widgets.HBox([file_picker, file_name]),widgets.HBox([img,txt]))

In [216]:
import torch
load_model = widgets.HBox([widgets.Label(value="Loading Model...")])
global model
global file_name
def load_model(self):
    # vbox.children = (widgets.HBox([file_picker, file_name]),load_model,widgets.HBox([img,txt]))
    model = "torch.load(file_name)"

In [217]:
import numpy as np
from traitlets import dlink
import ipyvolume as ipv
import ipywidgets as widgets
from IPython.display import display
from io import StringIO
from ipywidgets import Button, FileUpload, GridspecLayout


# widgets
file_picker = widgets.FileUpload(accept='*.pth', description='Upload your model:', layout=dict(width='auto', height='auto'))
file_picker.observe(load_model)
file_name = widgets.Text(disabled=True)

vbox = widgets.VBox([widgets.HTML(title_html, layout=widgets.Layout(margin='0 0 3em 0', max_width='800px')),
            widgets.HTML(description_html, layout=widgets.Layout(margin='3em 0 0 0', max_width='800px')),widgets.HBox([file_picker, file_name])])

img = Button(description='Image Model', button_style='primary', layout=dict(width='auto', height='auto'))
img.on_click(img_update)

txt = Button(description='Text Model', button_style='primary', layout=dict(width='auto', height='auto'))
txt.on_click(txt_update)

# links between widgets
dlink((file_picker, 'value'), (file_name, 'value'), get_name)
file_picker.observe(change_input, 'value')

vbox